# Scraper

In [37]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

def get_links(): 
    response = requests.get('https://en.wikipedia.org/wiki/List_of_language_names')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    # links = [link.get('href') for link in links if (link.get('title') is not None) and ('language' in link.get('title'))]
    links = [link.get('href') for link in links if (link.get('title') is not None)]
    # links = [link for link in links if link.endswith('_language')]
    links = ["https://en.wikipedia.org"+link for link in links]
    return links

In [90]:
countries_and_territories = requests.get('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_and_their_capitals_in_native_languages')
soup = BeautifulSoup(countries_and_territories.content, 'html.parser')

# how to specify multiple classes in find_all

table = soup.find('table', {'class': ['wikitable', 'sortable', 'jquery-tablesorter']})
regions = []
languages = []

for region in table.find('tbody').find_all('tr')[0:]:

    # region is the text in the first column 
    # official languages is the list of languages in the second column
    # regional languages is the list of languages in the third column 

    columns = region.find_all('td')

    region_name = columns[0].text
    print("region name: ", region_name)



    # for result in region.find_all('td'):
    #     ul_tag = result.find('ul')
    #     a_tag = result.find('a')
    #     print(ul_tag)
    #     print(a_tag)


    # print(languages)
    # print(type(result))
    # print(result)
    # off_lang = result[0]
    # print(result.text)


    # if r is not None:
    #     langs = []
    #     for language in region.find_all('td').find('<tr>'):

    #         # print(type(language))
    #         # print(language)

    #         if language.find('a') is not None:
    #             langs.append(language.find('a').get('title'))
    #         elif language.find('ul') is not None:
    #             for lang in language.find('ul').find_all('li'):
    #                 langs.append(lang.text)
    #         else:
    #             langs.append(None)   
    #     languages.update({r: langs})


# print(languages)

IndexError: list index out of range

In [39]:
unesco_languages = pd.read_csv('Extinct languages - DATA SUMMARY.csv')
unesco_languages

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered
...,...,...,...
2534,|'Auni,0,Extinct
2535,|Xam,0,Extinct
2536,ǁKu ǁ'e,0,Extinct
2537,ǁKx'au,0,Extinct


In [40]:


def find_index_of_headers(url):
    response        = requests.get(url)
    content         = response.content
    parser          = BeautifulSoup(content, 'html.parser')
    table           = parser.find('table', {'class': 'infobox'})
    included_headers = {}
    
    for i in range(0, len(table.find_all('th', {'class': 'infobox-label'}))):
        included_headers[unicodedata.normalize('NFKD', table.find_all('th', {'class': 'infobox-label'}).__getitem__(i).getText())] = i
    return included_headers


urls = get_links()
print(len(urls))
print(urls[:10])

urls = list(set(urls))

urls = [url for url in urls if url not in ['Official_language', 'Minority_language']]
urls = [url for url in urls if url.startswith('List_of_countries') == False]
urls = [url for url in urls if url.count(':') == 1]
urls = [url for url in urls if '?' not in url]

3540
['https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Wikipedia:Contents', 'https://en.wikipedia.org/wiki/Portal:Current_events', 'https://en.wikipedia.org/wiki/Special:Random', 'https://en.wikipedia.org/wiki/Wikipedia:About', 'https://en.wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us', 'https://en.wikipedia.orghttps://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Help:Introduction', 'https://en.wikipedia.org/wiki/Wikipedia:Community_portal']


In [42]:
urls = urls[0:100]

In [43]:
data = []
errors = []

for url in tqdm(urls): 
    # try: 
        lang = url.split('/')[-1]

        # print(requests.get(url).status_code)
        # print(url)
        response        = requests.get(url)
        content         = response.content
        parser          = BeautifulSoup(content, 'html.parser')
        table           = parser.find('table', {'class': 'infobox'})

        if table == None:
             print("No table found ", url)
             continue

        header_indices  = find_index_of_headers(url)

        # print(header_indices)

        infobox_data = table.find_all('td', {'class': 'infobox-data'})

        off_lang        = infobox_data[header_indices['Official language in']].text if 'Official language in' in header_indices else None
        rec_min_lang    = infobox_data[header_indices['Recognised minoritylanguage in']].text if 'Recognised minoritylanguage in' in header_indices else None
        # speakers        = infobox_data[header_indices['Speakers']].text             if 'Speakers' in header_indices else None
        iso3code        = infobox_data[header_indices['ISO 639-3']].text            if 'ISO 639-3' in header_indices else None

        if 'Speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Speakers']].text
        elif 'Native speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Native speakers']].text
        else:
            speakers = None

        if 'Region' in header_indices.keys():
            regions = infobox_data[header_indices['Region']].text
        elif 'Native Region' in header_indices.keys():
            regions = infobox_data[header_indices['Native Region']].text
        else:
            regions = None

        family          = [x for x in infobox_data[header_indices['Language family']].get_text('title').split('title') if x != "\n"]\
                            if 'Language family' in header_indices else None
        
        dialects        = [x for x in infobox_data[header_indices['Dialects']].get_text('title').split('title') if x != "\n"] \
                            if 'Dialects' in header_indices else None
        
        # print('Regions: ', regions)
        data.append((lang, family, dialects, iso3code, speakers, regions, off_lang, rec_min_lang))
    # except Exception as e: 
    #     errors.append((e, url))
    #     continue

df = pd.DataFrame(data, columns=['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions', 'off_lang', 'rec_min_lang'])


 28%|██▊       | 28/100 [00:39<01:37,  1.36s/it]

No table found  https://en.wikipedia.org/wiki/States_of_Mexico


 46%|████▌     | 46/100 [01:00<00:51,  1.06it/s]

No table found  https://en.wikipedia.org/wiki/List_of_countries_where_French_is_an_official_language


 74%|███████▍  | 74/100 [01:39<00:31,  1.22s/it]

...

 88%|████████▊ | 88/100 [02:01<00:14,  1.18s/it]

No table found  https://en.wikipedia.org/wiki/Akkad_(city)


 97%|█████████▋| 97/100 [02:13<00:03,  1.03s/it]

No table found  https://en.wikipedia.org/wiki/Kawthoolei


100%|██████████| 100/100 [02:17<00:00,  1.37s/it]


In [44]:
languages = df

In [45]:
languages.isna().sum().sort_values()/len(languages)

lang            0.000000
iso3code        0.542553
family          0.553191
speakers        0.585106
regions         0.627660
off_lang        0.808511
dialects        0.840426
rec_min_lang    0.872340
dtype: float64

In [46]:
languages[:20]

,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
0,Harari_language,"[Afro-Asiatic, Semitic, West Semitic, South Se...",None,har,"27,000 (2007 census)[2][3][4]",Harari Region,None,None
1,Assiniboine_language,"[Siouan, Western Siouan, (Core), Mississippi ...",None,asb,"150, 4.3% of ethnic population (2007)[1]","Saskatchewan, Canada\nMontana, United States",None,None
2,Mon%C3%A9gasque_dialect,"[Indo-European, Italic, Latino-Faliscan, Latin...",None,–,"(ca. 20 cited for 1950)[2]nearly extinct, but ...",None,None,Monaco
3,Bosnia_and_Herzegovina,None,None,None,None,None,None,None
4,Maharashtra,None,None,None,None,West India,None,None
5,Taa_language,"[Tuu, \nTaa–Lower Nossob, Taa]","[West ǃXoon (Nǀuǁʼen), ǃAma, East ǃXoon, Tsaas...",nmn,"2,500 (2011)[1]","Southern Ghanzi, northern Kgalagadi, western S...",None,None
6,Anglo-Saxon_runes,None,None,None,None,None,None,None
7,Cayuga_people,None,None,None,None,None,None,None
8,Republic_of_Chad,None,None,None,None,None,None,None
9,Tuva,None,None,None,None,None,None,None


In [51]:
unesco_languages

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered
...,...,...,...
2534,|'Auni,0,Extinct
2535,|Xam,0,Extinct
2536,ǁKu ǁ'e,0,Extinct
2537,ǁKx'au,0,Extinct


In [54]:
languages.merge(unesco_languages, left_on='lang', right_on='Name in English')
print(languages.columns)
languages

Index(['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions',
       'off_lang', 'rec_min_lang'],
      dtype='object')


,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
0,Harari_language,"[Afro-Asiatic, Semitic, West Semitic, South Se...",None,har,"27,000 (2007 census)[2][3][4]",Harari Region,None,None
1,Assiniboine_language,"[Siouan, Western Siouan, (Core), Mississippi ...",None,asb,"150, 4.3% of ethnic population (2007)[1]","Saskatchewan, Canada\nMontana, United States",None,None
2,Mon%C3%A9gasque_dialect,"[Indo-European, Italic, Latino-Faliscan, Latin...",None,–,"(ca. 20 cited for 1950)[2]nearly extinct, but ...",None,None,Monaco
3,Bosnia_and_Herzegovina,None,None,None,None,None,None,None
4,Maharashtra,None,None,None,None,West India,None,None
...,...,...,...,...,...,...,...,...
89,Tosk_Albanian,"[Indo-European, Albanoid, Albanian, Tosk]","[Northern Tosk, Southern Tosk, Arbëresh, Cham,...",als,1.8 million (2011 census)[1],"Albania, Kosovo, North Macedonia, Italy, Greec...",None,None
90,Yunnan_Province,None,None,None,None,None,None,None
91,East_New_Britain,None,None,None,None,None,None,None
92,Denmark,None,None,None,None,None,None,None
